## Data Wrangling for the DeepTropism
The goal of this notebook is to perform data wrangling on the original HIV-1 V3 loop sequences published on literature to have a consistent dataset for trainning.
